In [1]:
import time
import jwt
import requests
from numpy import dot
from numpy.linalg import norm


KEY = '83e5bc58555d8bac289e27bac50f8afc.Khk1JjCxb8MJN8Mi'

# 实际KEY，过期时间



def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split('.')
    except Exception as e:
        raise Exception('invalid apikey', e)

    payload = {
        'api_key': id,
        'exp': int(round(time.time() * 1000)) + exp_seconds * 1000,
        'timestamp': int(round(time.time() * 1000)),
    }
    return jwt.encode(
        payload,
        secret,
        algorithm='HS256',
        headers={'alg': 'HS256', 'sign_type': 'SIGN'},
    )

In [2]:
def ask_glm(content):
    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
      'Content-Type': 'application/json',
      'Authorization': generate_token(KEY, 1000)
    }

    data = {
        "model": "glm-3-turbo",
        "messages": [{"role": "user", "content": content}]
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()

In [3]:
import jieba

In [4]:
import json

In [5]:
questions = json.load(open('./submit_3.json'))

In [6]:
import pdfplumber

pdf = pdfplumber.open('./初赛训练数据集.pdf')

pdf_content = []

for idx, content in enumerate(pdf.pages):
    pdf_content.append(
        {
            'page_idx': str(idx + 1),
            'page_content': content.extract_text()
        }
    )

In [7]:
pdf_content[198]

{'page_idx': '199',
 'page_content': '驾驶辅助\n01设置FCTA灵敏度（无报警、低、中、高）。 01盲区（传感器视野受阻）。\n说明！ 02检测到的区域。\n□ FCTA和FCW是同一设置项，通过中央显示屏设置灵敏度时相当于 如果驾驶辅助系统传感器受限，可能会导致FCTA无法正常使用，请参\n同步设置了FCTA和FCW的灵敏度。 见驾驶辅助系统传感器章节。\n前方交叉路口预警系统局限性\n以下示例中，当两侧前雷达被周围的车辆以及障碍物遮挡，FCTA将无\n法及时检测到侧方来车。\n199'}

In [24]:
len(pdf_content)

354

In [28]:
for i in range(30):
    bm_page = int(questions[i]['reference'][5:]) - 1
    prompt = f"你是一个专业的汽车导购，请结合给定资料，回答以下问题。如果在资料中找不到答案，请回答：结合给定的资料，无法回答问题。\
        资料：来自手册的第：{bm_page + 1}页，内容为：{pdf_content[bm_page]['page_content']}；\
        问题： {questions[i]['question']}"
    answer = ask_glm(prompt)['choices'][0]['message']['content']
    questions[i]['answer'] = answer

In [29]:
questions[:30]

[{'question': '“前排座椅通风”的相关内容在第几页？',
  'answer': '结合给定的资料，无法回答问题。因为资料中并没有提供关于“前排座椅通风”的具体内容。',
  'reference': 'page_115'},
 {'question': '"关于车辆的儿童安全座椅固定装置，在哪一页可以找到相关内容？"',
  'answer': '关于车辆的儿童安全座椅固定装置的相关内容可以在第123页找到。',
  'reference': 'page_123'},
 {'question': '“打开前机舱盖”的相关信息在第几页？',
  'answer': '结合给定的资料，无法回答问题。因为资料中只提供了关于“关闭前机舱盖”的操作步骤和注意事项，并没有提供“打开前机舱盖”的相关信息。',
  'reference': 'page_308'},
 {'question': '“打开前机舱盖”这个操作在哪一页？',
  'answer': '打开前机舱盖这个操作的说明在第307页。',
  'reference': 'page_307'},
 {'question': '“查看行车记录仪视频”这一项内容在第几页？',
  'answer': '"查看行车记录仪视频"这一项内容在手册的第276页。',
  'reference': 'page_276'},
 {'question': '请问Lynk&Co领克汽车的事件数据记录系统（EDR）主要记录哪些信息？',
  'answer': 'Lynk&Co领克汽车的事件数据记录系统（EDR）主要记录以下信息：\n1. 车辆的行驶速度：表示事件发生时刻车辆的速度。\n2. 制动状态：表示事件发生时刻车辆是否进行制动。',
  'reference': 'page_12'},
 {'question': '问题：事件数据记录系统（EDR）中的数据是否可以被黑客利用进行恶意攻击？',
  'answer': '结合给定的资料，无法回答问题。',
  'reference': 'page_12'},
 {'question': '问题：在国家环保法要求下，哪些情况下需要对车辆进行报废处理？',
  'answer': '在国家环保法要求下，车辆在达到使用年限或者不能满足道路使用条件的情况下，需要进行报废处理。

In [30]:
for i in range(30, 61):
    bm_page = int(questions[i]['reference'][5:]) - 1
    prompt = f"你是一个专业的汽车导购，请结合给定资料，回答以下问题。如果在资料中找不到答案，请回答：结合给定的资料，无法回答问题。\
        资料：来自手册的第：{bm_page + 1}页，内容为：{pdf_content[bm_page]['page_content']}；\
        问题： {questions[i]['question']}"
    answer = ask_glm(prompt)['choices'][0]['message']['content']
    questions[i]['answer'] = answer

In [31]:
questions[30:61]

[{'question': '如何确保车辆在低速行驶或静止时，乘客安全地使用开门预警系统？',
  'answer': '确保车辆在低速行驶或静止时，乘客安全地使用开门预警系统的步骤如下：\n\n1. 确保车辆完全停稳。\n2. 当车速低于5km/h时，系统将自动激活。\n3. 使用车辆后视镜检查相邻两侧后方交通状况。\n4. 确保没有行人和其他车辆接近车辆后方。\n5. 如果一切安全，可以安全地打开车门。\n6. 如果系统检测到开门存在潜在的碰撞风险，外后视镜指示灯及氛围灯将闪烁，并发出声音警告。\n7. 在听到警告声或看到指示灯闪烁时，应停止开门动作，以避免发生碰撞事故。\n8. 驾驶员或乘客可以使用开启/关闭主驾座椅迎宾功能来调整座椅位置，以确保系统正常工作。\n\n遵循以上步骤，可以最大程度地确保在低速行驶或静止时，乘客能够安全地使用开门预警系统。',
  'reference': 'page_46'},
 {'question': '问题：主驾座椅迎宾功能在哪些情况下会自动停用？',
  'answer': '主驾座椅迎宾功能在驾驶员将座椅位置调整的很靠后时，会自动停用。',
  'reference': 'page_46'},
 {'question': '如何确保车辆的防盗系统正常工作？',
  'answer': '为确保车辆的防盗系统正常工作，请遵循以下步骤：\n\n1. 确保所有车门、尾门和机舱盖在锁止车辆前已经关闭。如果任何一侧车门未关闭，防盗系统将不会激活。\n2. 在使用遥控钥匙、蓝牙钥匙或Lynk&CoApp解锁车辆之前，确认车辆的防盗系统没有被触发。\n3. 如果在锁止车辆后10秒内未出现非法开启车门、机舱盖或尾门的行为，防盗系统将进入戒备状态。\n4. 使用合法的方式启动车辆，即确保遥控钥匙或蓝牙钥匙在车辆的检测范围内。\n5. 定期检查仪表显示屏，确保没有显示报警系统故障或其他相关的指示性信息。如果有任何异常，应立即联系Lynk&Co领克中心进行检查。\n6. 不要尝试使用Lynk&CoApp远程启动车辆来关闭已被触发的防盗系统。如果远程防盗系统被激活，尝试启动车辆时，仪表显示屏将显示车辆无法启动的信息。\n\n遵循这些步骤可以帮助确保您的车辆防盗系统正常工作，防止车辆或车辆内物品被盗。',
  'reference': 'page

In [32]:
for i in range(61, 90):
    bm_page = int(questions[i]['reference'][5:]) - 1
    prompt = f"你是一个专业的汽车导购，请结合给定资料，回答以下问题。如果在资料中找不到答案，请回答：结合给定的资料，无法回答问题。\
        资料：来自手册的第：{bm_page + 1}页，内容为：{pdf_content[bm_page]['page_content']}；\
        问题： {questions[i]['question']}"
    answer = ask_glm(prompt)['choices'][0]['message']['content']
    questions[i]['answer'] = answer

In [33]:
questions[61:90]

[{'question': '如何通过组合仪表设置项调整远近光灯？',
  'answer': '通过组合仪表设置项调整远近光灯的步骤如下：\n\n1. 将滚轮旋转到组合仪表设置项（行程信息、导航、多媒体、电话）的位置。\n2. 然后将滚轮旋转到组合仪表二级菜单设置项（不同组合仪表一级菜单设置项），找到近光灯启用的位置。\n3. 在近光灯启用位置，向前推动拨杆，即可启用近光灯。\n4. 要启用远光灯，同样将滚轮旋转到远光灯启用的位置，然后向前推动拨杆。\n\n注意：当组合仪表显示车门/安全带提醒界面时，需要先按下方向盘右侧的确认按键，再按下方向盘的相应按键，才能打开组合仪表设置菜单。',
  'reference': 'page_82'},
 {'question': '如何在行驶过程中快速切换远近光灯？',
  'answer': '在行驶过程中快速切换远近光灯的方法如下：\n\n1. 同时按下转向灯开关和危险警告灯开关，保持2秒钟。\n\n2. 当仪表显示屏上出现“近光”或“远光”字样时，迅速松开转向灯开关和危险警告灯开关。\n\n3. 此时，远近光灯会快速切换。\n\n注意：在某些情况下，如大雾、大雨、风雪、泥泞道路等，可能需要手动切换远光灯和近光灯。此时，请按照智能远近光控制系统的局限性进行操作。',
  'reference': 'page_85'},
 {'question': '如何确保前大灯自动开启/关闭功能及自动雨刮功能正常使用？',
  'answer': '为确保前大灯自动开启/关闭功能及自动雨刮功能正常使用，请确保雨量光线传感器没有被遮挡。',
  'reference': 'page_84'},
 {'question': '问题：如果雨量光线传感器被遮挡，如何确保前大灯自动开启/关闭功能和自动雨刮功能的正常使用？',
  'answer': '如果雨量光线传感器被遮挡，请确保及时清理遮挡物，以便传感器可以正常监测外界环境光照强度。只有当传感器正常工作，前大灯自动开启/关闭功能和自动雨刮功能才能正常使用。',
  'reference': 'page_84'},
 {'question': '如何确保大灯随动转向功能在适当的时候工作？',
  'answer': '为确保大灯随动转向功能在适当的时候工作，您需要遵循以下步骤：\n\n1. 确

In [34]:
for i in range(90, 121):
    bm_page = int(questions[i]['reference'][5:]) - 1
    prompt = f"你是一个专业的汽车导购，请结合给定资料，回答以下问题。如果在资料中找不到答案，请回答：结合给定的资料，无法回答问题。\
        资料：来自手册的第：{bm_page + 1}页，内容为：{pdf_content[bm_page]['page_content']}；\
        问题： {questions[i]['question']}"
    answer = ask_glm(prompt)['choices'][0]['message']['content']
    questions[i]['answer'] = answer

In [35]:
questions[90:121]

[{'question': '问题：如果天窗自动开启/关闭功能和防夹功能无法正常工作，驾驶员应该采取哪些措施来确保安全？',
  'answer': '如果天窗自动开启/关闭功能和防夹功能无法正常工作，驾驶员应该采取以下措施来确保安全：\n\n1. 不要尝试自行修理，以免造成伤害或损坏车辆。\n2. 建议立即联系Lynk&Co领克中心，由专业人员进行检查和维修。\n3. 在等待维修期间，尽量避免使用天窗，以确保行车安全。',
  'reference': 'page_112'},
 {'question': '如何通过中央显示屏调节前排座椅加热功能？',
  'answer': '通过中央显示屏调节前排座椅加热功能的步骤如下：\n\n1. 在中央显示屏中点击，进入驾驶员侧座椅加热控制界面。\n2. 点击、等指示箭头，调节选择的座椅功能。\n3. 设置驾驶员侧座椅加热强度及开关控制。\n4. 点击，进入副驾驶员侧座椅加热控制界面。\n5. 点击、等指示箭头，调节选择的座椅功能。\n6. 设置副驾驶员侧座椅加热强度及开关控制。',
  'reference': 'page_114'},
 {'question': '既然前排座椅加热可以通过中央显示屏进行调节，那么这个系统支持同时调节多个座椅的加热吗？',
  'answer': '结合给定的资料，无法回答问题。因为资料中并没有提供关于同时调节多个座椅加热的信息。',
  'reference': 'page_114'},
 {'question': '如何通过Lynk&CoApp设置前排座椅通风的时间？',
  'answer': '结合给定的资料，无法回答问题。资料中没有提供通过Lynk&CoApp设置前排座椅通风时间的步骤和信息。',
  'reference': 'page_115'},
 {'question': '请问如何设置前排座椅通风的自动关闭功能？',
  'answer': '结合给定的资料，无法回答问题。因为资料中并没有提及如何设置前排座椅通风的自动关闭功能。',
  'reference': 'page_116'},
 {'question': '请问前排头枕是如何为驾驶者的安全提供额外保护的？',
  'answer': '前排头枕为整体式头枕，是车辆安全系统的一部分，可以在某些碰撞事故中为驾驶

In [36]:
for i in range(121, 151):
    bm_page = int(questions[i]['reference'][5:]) - 1
    prompt = f"你是一个专业的汽车导购，请结合给定资料，回答以下问题。如果在资料中找不到答案，请回答：结合给定的资料，无法回答问题。\
        资料：来自手册的第：{bm_page + 1}页，内容为：{pdf_content[bm_page]['page_content']}；\
        问题： {questions[i]['question']}"
    answer = ask_glm(prompt)['choices'][0]['message']['content']
    questions[i]['answer'] = answer

In [37]:
questions[121:151]

[{'question': '如何设置方向盘转向助力模式以适应不同的驾驶模式？',
  'answer': '要设置方向盘转向助力模式以适应不同的驾驶模式，请按照以下步骤操作：\n\n1. 点击选择驾驶模式。\n2. 点击设置方向盘驾驶模式联动或自定义方向盘转向助力方式。\n3. 点击设置方向盘转向助力模式（轻便、舒适、运动）。\n\n开启方向盘助力与驾驶模式联动后，当驾驶员切换驾驶模式时，方向盘助力特性也会随着驾驶模式变化，形成适合驾驶员的方向盘助力。',
  'reference': 'page_148'},
 {'question': '问题：如果当前驾驶模式为运动模式，那么方向盘的转向助力方式是哪种？',
  'answer': '根据给定的资料，如果当前驾驶模式为运动模式，方向盘的转向助力方式是“运动”。',
  'reference': 'page_148'},
 {'question': '如何调整HUD的高度和亮度？',
  'answer': '调整HUD的高度和亮度，请按照以下步骤操作：\n\n1. 点击开启/关闭HUD，进入HUD设置界面。\n2. 在HUD设置项中，选择高度/亮度设置。\n3. 点击选择调节HUD高度亮度，选择相应设置项，该设置项将会高亮显示。\n4. 点击选择调节HUD角度，进行角度设置。\n5. 点击开启/关闭雪地模式，根据需要调节HUD亮度和高度。\n6. 点击复位HUD角度，将HUD角度重置。\n\n在调节HUD亮度和高度时，可以通过方向盘或中央显示屏进行操作。具体方法是：在组合仪表菜单界面，按下方向盘上的左/右按键进入界面，按下方向盘的上/下按键切换HUD设置项，然后按下方向盘上的确认按键选中HUD设置项。通过按下方向盘左、右区域，可以调节HUD亮度；按下方向盘上、下区域，可以调节HUD高度。',
  'reference': 'page_150'},
 {'question': '问题：HUD的高度和亮度设置对驾驶有什么具体影响？',
  'answer': 'HUD（抬头显示）的高度和亮度设置对驾驶有以下具体影响：\n\n1. **安全驾驶**：根据资料，驾驶前必须检查并确认HUD影像的位置和亮度不会妨碍安全驾驶。如果影像位置错误或亮度过低或过高，可能妨碍驾驶员的视野，增加发生事故的风险，甚至导致人身伤亡。\n\

In [38]:
for i in range(151, 181):
    bm_page = int(questions[i]['reference'][5:]) - 1
    prompt = f"你是一个专业的汽车导购，请结合给定资料，回答以下问题。如果在资料中找不到答案，请回答：结合给定的资料，无法回答问题。\
        资料：来自手册的第：{bm_page + 1}页，内容为：{pdf_content[bm_page]['page_content']}；\
        问题： {questions[i]['question']}"
    answer = ask_glm(prompt)['choices'][0]['message']['content']
    questions[i]['answer'] = answer

In [39]:
questions[151:181]

[{'question': 'LynkCo-Pilot系统在哪些情况下会提醒驾驶员接管车辆？',
  'answer': '根据给定资料，LynkCo-Pilot系统在出现故障时会提醒驾驶员接管车辆。具体来说，当系统出现问题时，组合仪表显示信息中的图标会显示灰色以提醒驾驶员。此外，在给定的资料中，没有提到其他会触发驾驶员接管车辆的情况。所以，根据资料，只有在LynkCo-Pilot系统出现故障时，驾驶员会被提醒接管车辆。',
  'reference': 'page_188'},
 {'question': '问题：LynkCo-Pilot系统在何种情况下会默认将跟车距离间隔设置为最远？',
  'answer': 'LynkCo-Pilot系统在每次启动车辆时会默认将跟车距离间隔设置为最远。',
  'reference': 'page_188'},
 {'question': '如何确保在使用高级智能驾驶辅助系统时保持安全距离？',
  'answer': '为确保在使用高级智能驾驶辅助系统时保持安全距离，请遵循以下指南：\n\n1. 设置合理的跟车距离间隔，以便在紧急情况下有足够的时间和空间做出反应。\n2. 始终保持对车辆的控制，并在必要时及时采取制动措施。\n3. 注意车速的变化，根据实际情况调整跟车距离间隔。在湿滑路面行驶时，请选择更大的跟车距离间隔。\n4. 当您踩下加速踏板时，车辆将暂时加速；松开加速踏板后，车速将慢慢减速至目标车速。\n5. 若您开启了“巡航自动限速辅助”功能，系统可以将当前道路限速牌上的限速值自动设定为目标车速。\n6. 如果您需要暂时停用高级智能驾驶辅助系统，可以按下方向盘上的相应按键或者踩下制动踏板，图标变为白色，LynkCo-Pilot进入准备模式。',
  'reference': 'page_184'},
 {'question': '问题：如何确保在各种路况下都能保持安全的跟车距离？',
  'answer': '为确保在各种路况下都能保持安全的跟车距离，请遵循以下指南：\n\n1. 根据车速调整跟车距离间隔。资料中指出，跟车距离间隔不是固定距离，与前车之间的实际距离会随车速变化而变化。因此，在高速行驶时应保持更大的跟车距离，以便有足够的时间和空间应对突发情况。\n\n2. 在湿滑路面行驶时，选择更大的跟车距离间隔。这

In [40]:
for i in range(181, 201):
    bm_page = int(questions[i]['reference'][5:]) - 1
    prompt = f"你是一个专业的汽车导购，请结合给定资料，回答以下问题。如果在资料中找不到答案，请回答：结合给定的资料，无法回答问题。\
        资料：来自手册的第：{bm_page + 1}页，内容为：{pdf_content[bm_page]['page_content']}；\
        问题： {questions[i]['question']}"
    answer = ask_glm(prompt)['choices'][0]['message']['content']
    questions[i]['answer'] = answer

In [41]:
questions[181:201]

[{'question': '问题：CMSF系统在哪些特定情况下可能无法正常工作？',
  'answer': 'CMSF系统在以下特定情况下可能无法正常工作：\n\n1. 只有Lynk&Co领克中心能对CMSF系统进行维修，否则可能会导致系统无法正常使用。\n2. 在湿滑路面上行驶时，车辆制动距离加长，这会削弱CMSF的防碰撞性能。\n3. 在强烈的外部照明条件下，驾驶员难以看到视觉警告信号，也可能影响风挡摄像头的探测功能。\n4. 如果乘客舱温度非常高，风挡摄像头可能会暂时关闭，系统也可能不会发出警告。\n5. 在恶劣天气下能见度降低，可能会降低系统功能。\n6. 风挡摄像头和雷达视野受限，可能会导致系统检测到车辆或行人的时间比预期延后或完全检测不到人或物。\n7. 如果驾驶员主动地对车辆进行操纵/制动，该系统可能不会做出反应或比预期延迟反应。',
  'reference': 'page_212'},
 {'question': '问题：在什么情况下，紧急转向避让辅助系统（EMA）可能会受限制？',
  'answer': '紧急转向避让辅助系统（EMA）可能会在以下情况下受限制：\n\n1. 车辆熄火或锁车后。\n2. 拖车连接时。\n3. 检测到相邻车道存在碰撞风险。\n4. 车辆存在穿越路沿的风险。\n5. 车身稳定控制系统故障、关闭或对车辆进行干预时。\n6. 生命探测功能只是一种辅助功能，您始终有责任确认好车内情况后再离开。\n7. 生命探测功能无法检测到体积过小的生命体。\n8. 请注意车辆周围网络环境，网络受限可能会导致报警信息无法发送或延迟发送。\n9. 遮挡物或运动的物体（例如：手机震动、风扇转动等）会干扰传感器的正常工作，可能会影响生命探测功能检测的准确性。',
  'reference': 'page_214'},
 {'question': '问题：在紧急情况下，EMA系统能够在多长时间内做出反应？',
  'answer': '结合给定的资料，无法回答问题。',
  'reference': 'page_187'},
 {'question': '如何评价216号驾驶辅助系统的性能和功能？',
  'answer': '结合给定的资料，无法回答问题。因为资料中没有提供关于216号驾驶辅助系统性能和功能的详细信息。',
  'refer

In [42]:
for i in range(201, 231):
    bm_page = int(questions[i]['reference'][5:]) - 1
    prompt = f"你是一个专业的汽车导购，请结合给定资料，回答以下问题。如果在资料中找不到答案，请回答：结合给定的资料，无法回答问题。\
        资料：来自手册的第：{bm_page + 1}页，内容为：{pdf_content[bm_page]['page_content']}；\
        问题： {questions[i]['question']}"
    answer = ask_glm(prompt)['choices'][0]['message']['content']
    questions[i]['answer'] = answer

In [43]:
questions[201:231]

[{'question': '请问在启用RPA功能后，如果遇到手机或遥控钥匙丢失的情况，应该如何操作？',
  'answer': '结合给定的资料，无法回答问题。因为资料中只介绍了如何使用RPA功能进行泊车，而没有涉及手机或遥控钥匙丢失的情况应如何操作。',
  'reference': 'page_233'},
 {'question': '在使用RPA功能时，哪些情况下应该暂停或避免使用该功能？',
  'answer': '在使用RPA功能时，应该暂停或避免使用该功能的情况包括：\n\n1. 确保车辆附近没有人员或障碍物，并时刻观察车辆附近环境，必要时暂停RPA功能。\n2. 车门、尾门、前舱盖未关闭或车辆未锁时，不要使用RPA功能。\n3. 方向盘未回正时，不要使用RPA功能。\n4. 避免在有坡度、积雪、积水、凹坑等道路使用RPA功能。\n5. 不要在车内使用RPA功能。\n6. 蓝牙信号不稳定时，RPA功能可能会退出，此时应暂停使用。\n7. 泊车辅助传感器仅在探测到障碍物时发出警告，可能不会发出警告信息或延迟发出警告信息，也可能发出不必要的警告信息。因此，不能完全依赖泊车辅助传感器对潜在碰撞进行警告，可能会造成严重人身伤害和财产损失。\n8. 在雨雪天气下受到强光直射，可能会影响泊车辅助摄像头和泊车辅助传感器等传感器的工作性能，导致RPA功能性能下降，可能会导致与周围障碍物发生挂蹭。\n9. RPA功能仅限用户在车辆周围环境可预测的停车场或车库使用。\n10. RPA功能的性能取决于泊车辅助传感器探测到的障碍物、车辆、人或其他物体与车辆的距离接近程度。',
  'reference': 'page_236'},
 {'question': '请问RPA功能在哪些情况下会暂停或退出？',
  'answer': '结合给定的资料，RPA功能会在以下情况下暂停或退出：\n\n1. 车辆附近有人员或障碍物，且未采取必要措施。\n2. 车门、尾门、前舱盖未关闭且车辆未锁。\n3. 方向盘未回正。\n4. 用户在车内尝试使用RPA功能。\n5. 蓝牙信号不稳定。\n6. 车辆处于有坡度、积雪、积水、凹坑等道路。\n7. 当用户从Lynk&CoApp上松开前进或后退按键后，存在时延导致车辆仍会运动一段距离。\n8. 泊车辅助传感器表面有覆盖物或脏污。\n9. 雨

In [44]:
for i in range(231, 261):
    bm_page = int(questions[i]['reference'][5:]) - 1
    prompt = f"你是一个专业的汽车导购，请结合给定资料，回答以下问题。如果在资料中找不到答案，请回答：结合给定的资料，无法回答问题。\
        资料：来自手册的第：{bm_page + 1}页，内容为：{pdf_content[bm_page]['page_content']}；\
        问题： {questions[i]['question']}"
    answer = ask_glm(prompt)['choices'][0]['message']['content']
    questions[i]['answer'] = answer

In [45]:
questions[231:261]

[{'question': '请问如何查看当前车辆所连接的Wi-Fi网络的信号强度？',
  'answer': '抱歉，根据给定资料，无法回答您的问题。资料中未提及如何查看当前车辆所连接的Wi-Fi网络的信号强度。',
  'reference': 'page_267'},
 {'question': '请问在您的汽车中，创建人脸信息时需要将车辆切换至哪个挡位？',
  'answer': '在您的汽车中，创建人脸信息时需要将车辆挡位切换至驻车挡（P）。',
  'reference': 'page_270'},
 {'question': '请问在Lynk&Co系统中，如何为我的车辆添加一个亲情账户？',
  'answer': '在Lynk&Co系统中，为您的车辆添加一个亲情账户的步骤如下：\n\n1. 在中央显示屏中点击“用户中心”，进入用户账户登录界面。\n2. 通过Lynk&CoApp扫描二维码登录账户。\n3. 点击“亲情账户”，进入亲情账户界面。\n4. 点击“+”，进入添加账户界面。\n5. 输入需添加的亲情人的手机号码。添加成功后，中央显示屏会显示添加成功相关文言消息。\n6. 创建过程中由于系统数据传输原因，进度可能会有一定延迟，请等待30分钟后再次查看。\n7. 如果需要更改车主账户请联系Lynk&Co领克中心。',
  'reference': 'page_272'},
 {'question': '请问Lynk&CoID是否支持同时在线多个账户？',
  'answer': '结合给定的资料，无法回答问题。',
  'reference': 'page_269'},
 {'question': '行车记录仪在遇到紧急情况时，会保存多长时间的视频？',
  'answer': '行车记录仪在遇到紧急情况时，会保存紧急视频触发前15秒和触发后15秒的视频。',
  'reference': 'page_274'},
 {'question': '请问行车记录仪在遇到紧急情况时，是否支持自动发送紧急录像到指定的手机号码或服务器？',
  'answer': '结合给定的资料，无法回答问题。',
  'reference': 'page_274'},
 {'question': '请问如何通过中央显示屏查看行车记录仪拍摄的照片？',
  'ans

In [46]:
for i in range(261, 291):
    bm_page = int(questions[i]['reference'][5:]) - 1
    prompt = f"你是一个专业的汽车导购，请结合给定资料，回答以下问题。如果在资料中找不到答案，请回答：结合给定的资料，无法回答问题。\
        资料：来自手册的第：{bm_page + 1}页，内容为：{pdf_content[bm_page]['page_content']}；\
        问题： {questions[i]['question']}"
    answer = ask_glm(prompt)['choices'][0]['message']['content']
    questions[i]['answer'] = answer

In [47]:
questions[261:291]

[{'question': '在最初的2000km行驶期间，发动机转速不应超过多少rpm？',
  'answer': '在最初的2000km行驶期间，发动机转速不应超过3000rpm。',
  'reference': 'page_306'},
 {'question': '在保养和维护过程中，需要注意哪些安全警告？',
  'answer': '在保养和维护过程中，需要注意以下安全警告：\n\n1. 安装钥匙盖时，请谨慎操作，不要损坏变形。\n2. 更换钥匙电池时，小心不要丢失电池或其他任何小部件。\n3. 务必干手操作，否则湿气可能导致电池生锈。\n4. 请勿触摸或移动遥控钥匙内的其他任何部件。\n5. 若您更换遥控钥匙电池后遥控钥匙仍无法正常使用，请及时联系Lynk&Co领克中心进行检查并处理。\n6. 切勿将车辆停放于温度过高或过低的环境中。\n7. 切勿将车辆停放于有高温热源的场所。\n8. 避免车辆长时间停放，每月至少使用一次车辆。\n9. 动力电池每月至少需要充满一次，以保证动力电池寿命。\n10. 车辆需要保持干燥，避免长时间在潮湿环境下停放，例如积水的停车场所等。\n11. 检查低压蓄电池的电缆是否正确连接且适当紧固。\n12. 非专业人士，请勿触碰、移动、拆解动力电池及相应的高压线路，或其他带有高压警示标识的部件。\n13. 当车辆或动力电池起火时，迅速离开车辆至安全距离，请使用灭火专用灭火器，使用水灭火或不正确灭火器灭火可能会导致触电。',
  'reference': 'page_304'},
 {'question': '请问如何检查发动机机油的压力警告灯是否正常工作？',
  'answer': '结合给定的资料，无法回答问题。因为资料中只提供了如何关闭前机舱盖和检查发动机机油的步骤，并没有提供如何检查发动机机油的压力警告灯是否正常工作的信息。',
  'reference': 'page_308'},
 {'question': '问题：为什么建议使用Lynk&Co领克原厂纯正“DOT4”制动液？',
  'answer': '结合给定的资料，无法回答问题。资料中没有提供为什么建议使用Lynk&Co领克原厂纯正“DOT4”制动液的具体原因。',
  'reference': 'page_310'},
 {'question': '问题

In [48]:
for i in range(291, 301):
    bm_page = int(questions[i]['reference'][5:]) - 1
    prompt = f"你是一个专业的汽车导购，请结合给定资料，回答以下问题。如果在资料中找不到答案，请回答：结合给定的资料，无法回答问题。\
        资料：来自手册的第：{bm_page + 1}页，内容为：{pdf_content[bm_page]['page_content']}；\
        问题： {questions[i]['question']}"
    answer = ask_glm(prompt)['choices'][0]['message']['content']
    questions[i]['answer'] = answer

In [49]:
questions[291:301]

[{'question': '在紧急情况下使用补胎套装时，应如何确保轮胎的气压处于适当范围？',
  'answer': '在紧急情况下使用补胎套装时，应确保轮胎的气压处于适当范围，可以通过以下步骤进行操作：\n\n1. 使用压力表（编号07）检查轮胎的当前气压。\n2. 根据轮胎型号和制造商的推荐，确定适当的气压水平。\n3. 如果有减压阀（编号06），可以调整减压阀来降低气压，以防止在充气过程中气压过高。\n4. 连接充气泵气管（编号09）到轮胎的气门嘴（编号03）。\n5. 开启充气泵（需插入电源，编号01）并缓慢充气，同时密切观察压力表，以确保气压不要超过推荐的 maximum level。\n6. 达到适当的气压后，关闭充气泵，并再次使用压力表检查气压，确保其准确无误。\n7. 完成充气后，确保补胎液罐（编号05）和所有设备妥善存放，远离儿童触及，并清洁任何接触到的皮肤。\n\n请注意，在整个过程中，应确保安全，如在道路上修补，应开启危险警告灯并放置三角警示牌（资料中未直接提及，但为常规安全操作）。同时，按照上述步骤操作，可以有效确保轮胎气压处于适当范围。',
  'reference': 'page_341'},
 {'question': '在紧急情况下，如何正确解锁并打开车辆的充电盖进行充电？',
  'answer': '在紧急情况下，请按照以下步骤正确解锁并打开车辆的充电盖进行充电：\n\n1. 确保车辆已经处于熄火状态。\n2. 长按仪表板上的解锁按钮，解锁充电盖。\n3. 打开充电盖，并取下充电插座保护盖。\n4. 将充电枪插入车辆充电插座，并确保充电枪被锁止。\n5. 充电时，仪表显示屏显示估计的剩余充电时间，充电接口插座中的LED指示灯闪绿光。',
  'reference': 'page_289'},
 {'question': '问题：在车辆开始充电时，应该先拆除哪个端子以防止火花产生？',
  'answer': '在车辆开始充电时，应该先拆除负极端子以防止火花产生。',
  'reference': 'page_344'},
 {'question': '如何正确安装汽车电子标识，以避免影响其功能？',
  'answer': '在安装汽车电子标识时，请确保不要被内后视镜安装支架、传感器支架等遮挡。同时，请不要在挡风玻璃上粘贴金属

In [50]:
with open('./submit_4.json', 'w', encoding='utf8') as f:
    json.dump(questions, f, ensure_ascii=False, indent=4)